In [48]:
#Assignment 3
import cv2
import numpy as np
import time
import math




In [49]:
#cap = cv2.VideoCapture("https://192.168.1.76:8080/video") 
cap = cv2.VideoCapture(0) 


def angle(line):
    x1, y1, x2, y2=line
    return math.degrees(math.atan2(y2 - y1, x2 - x1))

def block(angle1, angle2, threshold=10):
    return abs(abs(angle1 - angle2) - 90) <= threshold

def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def transform(image, pts, output_size=(500, 500)):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    dst = np.array([
        [0, 0],
        [output_size[0] - 1, 0],
        [output_size[0] - 1, output_size[1] - 1],
        [0, output_size[1] - 1]
    ], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, output_size)

    return warped

def find_intersections(lines):
    intersections = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            line1 = lines[i][0]
            line2 = lines[j][0]

            x1, y1, x2, y2 = line1
            x3, y3, x4, y4 = line2

            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            # Überprüfe, ob die Linien nicht parallel sind
            if den != 0:
                px = int(((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / den)
                py = int(((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / den)
                
                cv2.circle(frame,(px,py), 5,(255,0,0), -1)
                
                intersections.append((int(px), int(py)))

    return intersections

In [50]:
while True:
    ret, frame = cap.read()
    gaus = cv2.GaussianBlur(frame, (5, 5), 0)
    gaus2 = cv2.GaussianBlur(gaus, (5, 5), 0)
    gaus3 = cv2.dilate(gaus2, (5, 5))
    edges = cv2.Canny(gaus3, 100, 200)
    lines1 = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=200, maxLineGap=200)

    if lines1 is not None:
        lines2 = np.array(sorted(lines1, key=lambda line: np.linalg.norm(line[0, :2] - line[0, 2:])))
        lines = lines2[:4]

        if lines is not None:
            i = 0

            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                i=i+1

                if i > 3:
                          
                    intersections=find_intersections(lines)
                    
                    if len(intersections)>3:
                        scan = transform(frame,np.array(intersections, dtype="float32"))
                        
    if scan is not None:
        cv2.imshow('scan',scan) 
                   
    # cv2.imshow('canny', edges)           
    cv2.imshow('frame', frame)
   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [51]:
intersections


[(-733102, -50492),
 (514, 453),
 (1085186, 75778),
 (514, 102),
 (-736, 16),
 (514, 103)]